In [24]:
# Install necessary libraries
# pip install yfinance yahooquery

from datetime import datetime, timedelta
from yahooquery import search
import yfinance as yf
import pandas as pd
import datetime
import os

def get_stock_data(company_name):
    # Map company name to ticker symbol
    ticker_symbol = get_ticker_symbol(company_name)
    if not ticker_symbol:
        print(f"Can't get ticker symbol for {company_name}")
        return

    # Fetch data from 2019-01-01 to today
    start_date = '2019-01-01'
    end_date = datetime.datetime.today().strftime('%Y-%m-%d')
    data = yf.download(ticker_symbol, start=start_date, end=end_date)
    
    # Check if data is available
    if data.empty:
        print(f"No data available for {company_name} ({ticker_symbol})")
        return

    # Check if 'Adj Close' column exists; if not, use 'Close' as a fallback
    if 'Adj Close' in data.columns:
        data = data[['Adj Close']]
    elif 'Close' in data.columns:
        data = data[['Close']]
        data.rename(columns={'Close': 'Adj Close'}, inplace=True)
    else:
        print(f"No 'Adj Close' or 'Close' data available for {company_name} ({ticker_symbol})")
        return
    
    # Format date and reset index
    data.reset_index(inplace=True)
    data['Date'] = data['Date'].dt.strftime('%Y-%m-%d')
    # print(data)

    # Create folder if it doesn't exist
    # folder_path = 'StockPrice'
    # os.makedirs(folder_path, exist_ok=True)

    # Save data as CSV with company name in filename
    # csv_filename = f"{folder_path}/{company_name}_stock_data.csv"
    # data.to_csv(csv_filename, index=False)
    # print(f"Data saved to {csv_filename}")

    # Retrieve the most recent adjusted close price if real-time data is unavailable
    # ticker = yf.Ticker(ticker_symbol)
    # history = ticker.history(period='1d')
    # if 'Adj Close' in history.columns:
    #     current_price = history['Adj Close'].iloc[-1]
    # elif 'Close' in history.columns:
    #     current_price = history['Close'].iloc[-1]
    # else:
    #     current_price = None

    # if current_price is not None:
    #     print(f"{company_name} ({ticker_symbol}) most recent Adj Close: ${current_price}")
    # else:
    #     print(f"No recent 'Adj Close' or 'Close' price available for {company_name} ({ticker_symbol})")
    return data

def get_ticker_symbol(company_name):
    # Use yahooquery's search function to get the ticker symbol
    result = search(company_name)
    if 'quotes' in result and len(result['quotes']) > 0:
        # Return the first matching ticker symbol
        ticker_symbol = result['quotes'][0]['symbol']
        return ticker_symbol
    else:
        return None


In [ ]:
# Example usage
company_name = 'Singtel'
data = get_stock_data(company_name)
data.iloc[1:, :]

[*********************100%***********************]  1 of 1 completed

Price                     Adj Close
Ticker                       Z74.SI
Date                               
2019-01-01 00:00:00+00:00  2.420833
2019-01-02 00:00:00+00:00  2.379522
2019-01-03 00:00:00+00:00  2.362997
2019-01-04 00:00:00+00:00  2.396046
2019-01-07 00:00:00+00:00  2.429095
...                             ...
2024-11-04 00:00:00+00:00  3.120000
2024-11-05 00:00:00+00:00  3.160000
2024-11-06 00:00:00+00:00  3.210000
2024-11-07 00:00:00+00:00  3.200000
2024-11-08 00:00:00+00:00  3.250000

[1473 rows x 1 columns]


Price,Date,Adj Close
Ticker,,Z74.SI
0,2019-01-01,2.420833
1,2019-01-02,2.379522
2,2019-01-03,2.362997
3,2019-01-04,2.396046
4,2019-01-07,2.429095
...,...,...
1468,2024-11-04,3.120000
1469,2024-11-05,3.160000
1470,2024-11-06,3.210000


In [16]:
data.reset_index(inplace=True)

In [15]:
import plotly.express as px

if data is not None:
	fig = px.line(data, x='Date', y='Adj Close', title=f"{company_name} Stock Price")
	fig.show()
else:
	print("No data to plot.")

ValueError: Data must be 1-dimensional, got ndarray of shape (1473, 1) instead

In [17]:
data

Price,index,Date,Adj Close
Ticker,,,Z74.SI
0,0,2019-01-01,2.420833
1,1,2019-01-02,2.379522
2,2,2019-01-03,2.362997
3,3,2019-01-04,2.396046
4,4,2019-01-07,2.429095
...,...,...,...
1468,1468,2024-11-04,3.120000
1469,1469,2024-11-05,3.160000
1470,1470,2024-11-06,3.210000
